In [5]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Step 1: Load and preprocess the BMI dataset
def load_bmi_data():
    df = pd.read_csv('/Users/dhrumilpatel/Downloads/bmi.csv') 
    return df

def preprocess_bmi_data(df):
    # Clean column names
    df.columns = df.columns.str.strip()

    # Print unique BMI Classes for debugging
    print("Unique BMI Classes before encoding:", df['BmiClass'].unique())

    # Convert categorical 'BmiClass' to numeric values
    label_encoder = LabelEncoder()
    df['BmiClass'] = label_encoder.fit_transform(df['BmiClass'])

    # Print mapping of original BMI Classes to encoded values
    class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print("BMI Class Mapping:", class_mapping)

    # Features and target variable
    X = df[['Age', 'Weight', 'Height']]  # Features
    y = df['BmiClass']  # Target variable

    return X, y, label_encoder, class_mapping


# Step 2: Train the BMI model
def train_bmi_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

    joblib.dump(model, 'bmi_model.pkl')
    print("Model saved as 'bmi_model.pkl'")
    return model


# Step 3: Load and preprocess the food dataset for recommendations
def load_food_data():
    food_df = pd.read_csv('./cleaned_dataset.csv')
    return food_df

def recommend_diet_plan(user_bmi_class, food_df, class_mapping):
    print(f"\nUser BMI Class (Encoded): {user_bmi_class}")

    # Reverse mapping for better readability
    bmi_class_label = {v: k for k, v in class_mapping.items()}
    user_bmi_class_label = bmi_class_label[user_bmi_class]
    print(f"User BMI Class Label: {user_bmi_class_label}")

    # Logic based on BMI Class
    if user_bmi_class_label == "Underweight":
        print("Detected Underweight")
        recommended_food = food_df[food_df['Caloric Value'] > 300]
    elif user_bmi_class_label == "Normal weight":
        print("Detected Normal Weight")
        user_choice = input("Do you want to gain weight? (yes/no): ").strip().lower()
        if user_choice == "yes":
            recommended_food = food_df[food_df['Caloric Value'] > 300]
        else:
            recommended_food = food_df  # Balanced diet
    elif user_bmi_class_label in ["Overweight", "Obese class 1", "Obese class 2", "Obese class 3"]:
        print(f"Detected {user_bmi_class_label}")
        recommended_food = food_df[food_df['Caloric Value'] < 200]
    else:
        print("Unexpected BMI Class, no recommendations available.")
        return

    # Ensure that recommended_food is not empty
    if recommended_food.empty:
        print("No foods match your criteria. Please adjust your BMI class or filter.")
        return

    # Randomly sample meals
    breakfast = recommended_food.sample(1)
    lunch = recommended_food.sample(1)
    dinner = recommended_food.sample(1)

    print(f"\nBreakfast: {breakfast['food'].values[0]} - Caloric Value: {breakfast['Caloric Value'].values[0]}")
    print(f"Lunch: {lunch['food'].values[0]} - Caloric Value: {lunch['Caloric Value'].values[0]}")
    print(f"Dinner: {dinner['food'].values[0]} - Caloric Value: {dinner['Caloric Value'].values[0]}")


# Step 4: Main function
def main():
    # Load and preprocess BMI data
    bmi_data = load_bmi_data()
    X, y, label_encoder, class_mapping = preprocess_bmi_data(bmi_data)

    # Train the model
    bmi_model = train_bmi_model(X, y)

    # Get user input
    print("\nEnter your details for BMI prediction:")
    age = int(input("Age: "))
    weight = float(input("Weight (kg): "))
    height = float(input("Height (cm): "))

    # Validate inputs
    if weight <= 0 or height <= 0 or age <= 0:
        print("Invalid input values. Please enter positive values.")
        return

    # BMI range check
    if age < 10 or age > 100:
        print("Please enter a realistic age (10-100).")
        return
    if weight < 10 or weight > 300:
        print("Please enter a realistic weight (10-300 kg).")
        return
    if height < 50 or height > 250:
        print("Please enter a realistic height (50-250 cm).")
        return

    # Predict BMI Class
    user_input = np.array([[age, weight, height]])
    bmi_class_prediction = bmi_model.predict(user_input)

    print(f"\nPredicted BMI Class: {label_encoder.inverse_transform(bmi_class_prediction)[0]}")

    # Load food data and recommend diet plan
    food_data = load_food_data()
    recommend_diet_plan(bmi_class_prediction[0], food_data, class_mapping)


# Run the main function
if __name__ == '__main__':
    main()


Unique BMI Classes before encoding: ['Obese Class 1' 'Overweight' 'Underweight' 'Obese Class 2'
 'Obese Class 3' 'Normal Weight']
BMI Class Mapping: {'Normal Weight': 0, 'Obese Class 1': 1, 'Obese Class 2': 2, 'Obese Class 3': 3, 'Overweight': 4, 'Underweight': 5}
Model Accuracy: 95.07%
Model saved as 'bmi_model.pkl'

Enter your details for BMI prediction:


Age:  38
Weight (kg):  74.8
Height (cm):  171



Predicted BMI Class: Normal Weight

User BMI Class (Encoded): 0
User BMI Class Label: Normal Weight
Unexpected BMI Class, no recommendations available.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
